# Efecto Croma (pantallas verdes)

El efecto Croma (pantalla verde) es una técnica utilizada en la postproducción
de video y fotografía para combinar dos imágenes o videos en una sola.
Seguramente has visto cómo se graban películas de ciencia ficción o programas de
televisión donde los actores interactúan con un fondo que no está presente; esto
se logra grabando a los actores frente a una pantalla verde y luego reemplazando
el color verde por el fondo deseado.

En este cuaderno vamos a implementar una versión simplificada del efecto Croma
utilizando la técnica de umbralización en scikit-image.

In [ ]:
import skimage as ski
import matplotlib.pyplot as plt
import numpy as np

Primero carguemos una imagen de un actor frente a una pantalla verde y una
imagen de un fondo que queremos utilizar.

In [ ]:
ruta_primer_plano = 'pantallas/Girl_in_front_of_a_green_background.jpg'
ruta_fondo = 'pantallas/enchanted_forest_background.jpg'

primer_plano = ski.util.img_as_float64(ski.io.imread(ruta_primer_plano))
fondo = ski.util.img_as_float64(ski.io.imread(ruta_fondo))
# Reescalar el fondo para que tenga la misma altura que el primer plano
fondo = ski.transform.resize(fondo, (primer_plano.shape[0], fondo.shape[1]))

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ski.io.imshow(primer_plano, ax=ax[0])
ski.io.imshow(fondo, ax=ax[1])

ax[0].set_title('Primer plano')
ax[0].axis('off')
ax[1].set_title('Fondo')
ax[1].axis('off')
fig.tight_layout()

In [ ]:
llave = np.array([0.0, 1.0, 0.0], dtype=np.float64)
distancia_min = 0.6
distancia_max = 0.7

# Calcular la distancia euclidiana entre el color de cada pixel del primer plano
# y el color de la llave
distancia = np.linalg.norm(primer_plano - llave, axis=2)

# Escalar la distancia y recortar (clip) para que la distancia máxima sea 1
# Luego, invertir la distancia para que los pixeles más cercanos a la llave
# tengan un valor más alto
distancia = 1 - np.clip(
    (distancia - distancia_min) / (distancia_max - distancia_min), 0, 1
)

ski.io.imshow(distancia, cmap="gray")

In [ ]:
# La distancia actúa como una máscara para combinar el primer plano y el fondo
# de forma suave. Vamos a centrar la imagen del primer plano en el fondo
x_0 = (fondo.shape[1] - primer_plano.shape[1]) // 2
x_1 = x_0 + primer_plano.shape[1]
y_0 = (fondo.shape[0] - primer_plano.shape[0]) // 2
y_1 = y_0 + primer_plano.shape[0]
combinado = fondo.copy()

# Aplicar la máscara al fondo
combinado[y_0:y_1, x_0:x_1] *= distancia[:, :, None]
fondo_enmascarado = combinado.copy()

# Combinar el primer plano y el fondo
primer_plano_enmascarado = primer_plano * (1 - distancia[:, :, None])
combinado[y_0:y_1, x_0:x_1] += primer_plano_enmascarado

fig, ax = plt.subplots(6, 1, figsize=(10, 25))
ax[0].imshow(primer_plano)
ax[0].set_title('Primer plano')
ax[1].imshow(fondo)
ax[1].set_title('Fondo')
ax[2].imshow(distancia, cmap='gray')
ax[2].set_title('Máscara')
ax[3].imshow(fondo_enmascarado)
ax[3].set_title('Fondo con máscara')
ax[4].imshow(primer_plano_enmascarado)
ax[4].set_title('Primer plano con máscara')
ax[5].imshow(combinado)
ax[5].set_title('Resultado final')

for ax_i in ax:
    ax_i.axis('off')